In [1]:
import numpy as np
import pandas as pd
from hurricanes.calc import convert_lon_180_to_360
import os
from pathlib import Path
import xarray as xr

glider = "ng645-20210613T0000"
days_offset = 2
region = [-98, -80, 18, 31]

# Set main path of data and plot location
root_dir = Path.home() / "Documents"

# Paths to data sources
path_data = (root_dir / "data") # create data path
path_gofs = "https://tds.hycom.org/thredds/dodsC/GLBy0.08/expt_93.0"
path_gliders = (path_data / "gliders")
path_impact = (path_data / "impact_metrics")
path_impact_calculated = path_impact / "calculated"
path_impact_model = path_impact / "models"

# Read glider dataframe output from erddap
glider_pickle = path_gliders / f"{glider}_data.pkl"

try:
    df = pd.read_pickle(glider_pickle)
except FileNotFoundError:
    # Download glider data from erddap with dataset id
    df = get_glider_by_id(glider)
    df.to_pickle(glider_pickle) # Save glider data to pickle file
    
df = df.reset_index()
df = df.rename({
    "time (UTC)": "time",
    "longitude (degrees_east)": "lon",
    "latitude (degrees_north)": "lat",
    "pressure (decibar)": "pressure",
    "temperature (degrees_C)": "temp",
    "depth (m)": "depth",
    "salinity (1)": "salinity",
    "conductivity (mS cm-1)": "conductivity",
    "density (kg m-3)": "density",
}, axis=1)

df = df.set_index("time").sort_index()
dstr = "%Y-%m-%d"
t0 = df.index.min().strftime(dstr)
t1 = df.index.max().strftime(dstr)
tdf = df.groupby(level=0).first()
glon = tdf["lon"]
glat = tdf["lat"]
region = [
    convert_lon_180_to_360(-98),
    convert_lon_180_to_360(-80),
    18,
    31
    ]

if days_offset:
    gtime = tdf.index.shift(days_offset, freq='D')
else:
    gtime = tdf.index 

In [2]:
drange = pd.date_range(t0, t1, freq='6H')
drange

DatetimeIndex(['2021-06-13 00:00:00', '2021-06-13 06:00:00',
               '2021-06-13 12:00:00', '2021-06-13 18:00:00',
               '2021-06-14 00:00:00', '2021-06-14 06:00:00',
               '2021-06-14 12:00:00', '2021-06-14 18:00:00',
               '2021-06-15 00:00:00', '2021-06-15 06:00:00',
               ...
               '2021-09-21 18:00:00', '2021-09-22 00:00:00',
               '2021-09-22 06:00:00', '2021-09-22 12:00:00',
               '2021-09-22 18:00:00', '2021-09-23 00:00:00',
               '2021-09-23 06:00:00', '2021-09-23 12:00:00',
               '2021-09-23 18:00:00', '2021-09-24 00:00:00'],
              dtype='datetime64[ns]', length=413, freq='6H')

In [3]:
gofs = xr.open_dataset(
    path_gofs, 
    drop_variables=['tau', 'water_temp_bottom', 'salinity_bottom', 'water_u_bottom', 'water_v_bottom']
    ).rename(
    {'surf_el': 'sea_surface_height',
     'water_temp': 'temperature',
     'water_u': 'u',
     'water_v': 'v'
     }
    )
gofs

<xarray.Dataset>
Dimensions:             (lat: 4251, lon: 4500, time: 9801, depth: 40)
Coordinates:
  * lat                 (lat) float64 -80.0 -79.96 -79.92 ... 89.92 89.96 90.0
  * lon                 (lon) float64 0.0 0.07996 0.16 ... 359.8 359.8 359.9
  * time                (time) datetime64[ns] 2018-12-04T12:00:00 ... 2022-04...
  * depth               (depth) float64 0.0 2.0 4.0 6.0 ... 3e+03 4e+03 5e+03
Data variables:
    sea_surface_height  (time, lat, lon) float32 ...
    temperature         (time, depth, lat, lon) float32 ...
    salinity            (time, depth, lat, lon) float32 ...
    u                   (time, depth, lat, lon) float32 ...
    v                   (time, depth, lat, lon) float32 ...
Attributes:
    classification_level:      UNCLASSIFIED
    distribution_statement:    Approved for public release. Distribution unli...
    downgrade_date:            not applicable
    classification_authority:  not applicable
    institution:               Fleet Numerical Meteorology and Oceanography C...
    source:                    HYCOM archive file
    history:                   archv2ncdf2d
    comment:                   p-grid
    field_type:                instantaneous
    Conventions:               CF-1.6 NAVO_netcdf_v1.1

In [4]:
tmp = gofs.sel(
    time=slice(pd.to_datetime(t0), pd.to_datetime(t1)),
    depth=slice(0, 1000), 
    lon=slice(region[0], region[1]), 
    lat=slice(region[2], region[3])
    )#.chunk({"time": 100, 'lon': 113, 'lat': 163})
tmp

<xarray.Dataset>
Dimensions:             (lat: 326, lon: 226, time: 825, depth: 33)
Coordinates:
  * lat                 (lat) float64 18.0 18.04 18.08 ... 30.92 30.96 31.0
  * lon                 (lon) float64 262.0 262.1 262.2 ... 279.8 279.9 280.0
  * time                (time) datetime64[ns] 2021-06-13 ... 2021-09-24
  * depth               (depth) float64 0.0 2.0 4.0 6.0 ... 800.0 900.0 1e+03
Data variables:
    sea_surface_height  (time, lat, lon) float32 ...
    temperature         (time, depth, lat, lon) float32 ...
    salinity            (time, depth, lat, lon) float32 ...
    u                   (time, depth, lat, lon) float32 ...
    v                   (time, depth, lat, lon) float32 ...
Attributes:
    classification_level:      UNCLASSIFIED
    distribution_statement:    Approved for public release. Distribution unli...
    downgrade_date:            not applicable
    classification_authority:  not applicable
    institution:               Fleet Numerical Meteorology and Oceanography C...
    source:                    HYCOM archive file
    history:                   archv2ncdf2d
    comment:                   p-grid
    field_type:                instantaneous
    Conventions:               CF-1.6 NAVO_netcdf_v1.1

In [5]:
gofs = gofs.sel(
    time=drange,
    depth=slice(0, 1000), 
    lon=slice(region[0], region[1]), 
    lat=slice(region[2], region[3])
    )
gofs

<xarray.Dataset>
Dimensions:             (lat: 326, lon: 226, time: 413, depth: 33)
Coordinates:
  * lat                 (lat) float64 18.0 18.04 18.08 ... 30.92 30.96 31.0
  * lon                 (lon) float64 262.0 262.1 262.2 ... 279.8 279.9 280.0
  * time                (time) datetime64[ns] 2021-06-13 ... 2021-09-24
  * depth               (depth) float64 0.0 2.0 4.0 6.0 ... 800.0 900.0 1e+03
Data variables:
    sea_surface_height  (time, lat, lon) float32 ...
    temperature         (time, depth, lat, lon) float32 ...
    salinity            (time, depth, lat, lon) float32 ...
    u                   (time, depth, lat, lon) float32 ...
    v                   (time, depth, lat, lon) float32 ...
Attributes:
    classification_level:      UNCLASSIFIED
    distribution_statement:    Approved for public release. Distribution unli...
    downgrade_date:            not applicable
    classification_authority:  not applicable
    institution:               Fleet Numerical Meteorology and Oceanography C...
    source:                    HYCOM archive file
    history:                   archv2ncdf2d
    comment:                   p-grid
    field_type:                instantaneous
    Conventions:               CF-1.6 NAVO_netcdf_v1.1

In [6]:
gofs = gofs[['temperature', 'salinity']]#.chunk({"time": 50, 'lon': 113, 'lat': 163})
gofs

<xarray.Dataset>
Dimensions:      (time: 413, depth: 33, lat: 326, lon: 226)
Coordinates:
  * lat          (lat) float64 18.0 18.04 18.08 18.12 ... 30.88 30.92 30.96 31.0
  * lon          (lon) float64 262.0 262.1 262.2 262.2 ... 279.8 279.9 280.0
  * time         (time) datetime64[ns] 2021-06-13 ... 2021-09-24
  * depth        (depth) float64 0.0 2.0 4.0 6.0 8.0 ... 700.0 800.0 900.0 1e+03
Data variables:
    temperature  (time, depth, lat, lon) float32 ...
    salinity     (time, depth, lat, lon) float32 ...
Attributes:
    classification_level:      UNCLASSIFIED
    distribution_statement:    Approved for public release. Distribution unli...
    downgrade_date:            not applicable
    classification_authority:  not applicable
    institution:               Fleet Numerical Meteorology and Oceanography C...
    source:                    HYCOM archive file
    history:                   archv2ncdf2d
    comment:                   p-grid
    field_type:                instantaneous
    Conventions:               CF-1.6 NAVO_netcdf_v1.1

In [ ]:
# GOFS: Select the time, lon, and lat nearest the glider
transect = gofs.sel(
    time=xr.DataArray(gtime, dims='point'),
    lon=xr.DataArray(convert_lon_180_to_360(glon.values), dims="point"),
    lat=xr.DataArray(glat.values, dims='point'),
    method='nearest'
    )
transect

In [8]:
clon = convert_lon_180_to_360(glon.values)
clon

array([265.40805817, 265.41266632, 265.41628265, ..., 271.83975983,
       271.83586121, 271.83279419])

In [9]:
# GOFS: Select the time, lon, and lat nearest the glider
tgofs = gofs.sel(
    time=gtime[0],
    lon=clon[0],
    lat=glat[0],
    method='nearest'
)
tgofs

<xarray.Dataset>
Dimensions:      (depth: 33)
Coordinates:
    lat          float64 27.64
    lon          float64 265.4
    time         datetime64[ns] 2021-06-15T06:00:00
  * depth        (depth) float64 0.0 2.0 4.0 6.0 8.0 ... 700.0 800.0 900.0 1e+03
Data variables:
    temperature  (depth) float32 ...
    salinity     (depth) float32 ...
Attributes:
    classification_level:      UNCLASSIFIED
    distribution_statement:    Approved for public release. Distribution unli...
    downgrade_date:            not applicable
    classification_authority:  not applicable
    institution:               Fleet Numerical Meteorology and Oceanography C...
    source:                    HYCOM archive file
    history:                   archv2ncdf2d
    comment:                   p-grid
    field_type:                instantaneous
    Conventions:               CF-1.6 NAVO_netcdf_v1.1

In [10]:
%timeit tgofs.load()

21.6 µs ± 394 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [11]:
tgofs

<xarray.Dataset>
Dimensions:      (depth: 33)
Coordinates:
    lat          float64 27.64
    lon          float64 265.4
    time         datetime64[ns] 2021-06-15T06:00:00
  * depth        (depth) float64 0.0 2.0 4.0 6.0 8.0 ... 700.0 800.0 900.0 1e+03
Data variables:
    temperature  (depth) float32 29.42 29.39 28.82 28.54 ... nan nan nan nan
    salinity     (depth) float32 34.29 34.29 34.31 34.33 ... nan nan nan nan
Attributes:
    classification_level:      UNCLASSIFIED
    distribution_statement:    Approved for public release. Distribution unli...
    downgrade_date:            not applicable
    classification_authority:  not applicable
    institution:               Fleet Numerical Meteorology and Oceanography C...
    source:                    HYCOM archive file
    history:                   archv2ncdf2d
    comment:                   p-grid
    field_type:                instantaneous
    Conventions:               CF-1.6 NAVO_netcdf_v1.1

In [ ]:
transect.load()

In [ ]:
save_file = path_impact_model / f"{glider}_gofs_{days_offset}day_offset_data.nc"
test.to_netcdf(save_file)

In [ ]:
# gldf = [transect.sel(point=t) for t in transect.point]
# # gldf

In [ ]:
# gofs_transect = xr.concat(gldf, dim="point")
# gofs_transect

In [ ]:
from hurricanes.calc import convert_lon_180_to_360
region = [convert_lon_180_to_360(-98),
          convert_lon_180_to_360(-80),
          18,
          31]
region

In [ ]:
ds = ds.sel(
    time=slice(pd.Timestamp(2021, 6, 1), pd.Timestamp(2021, 10, 1)),
    depth=slice(0, 1000), 
    lon=slice(region[0], region[1]), 
    lat=slice(region[2], region[3])
)
ds

In [ ]:
tds = ds.chunk({"time": 100, 'lon': 113, 'lat': 163})
tds